# Text classification: classify spam email and non-spam email by several machine learning models

In [1]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_predict
import pmdarima
from sklearn.metrics import mean_absolute_error 
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt

import numpy as np
from numpy import *
import pandas as pd
from pandas import MultiIndex, Int16Dtype
import matplotlib.pyplot as plt

import os
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from textwrap import wrap
import inspect
import itertools

import inspect
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xlsxwriter
import xlwings as xw
from copy import copy
import scipy.stats as ss
import math
from statsmodels.stats.stattools import durbin_watson
from scipy.optimize import curve_fit
import time
from IPython import display as ICD
import warnings
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill 
import re

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import callbacks
from keras.layers import Dropout
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import scikeras
from scikeras.wrappers import KerasRegressor
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split


from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from nltk.classify.util import accuracy
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [3]:
directory='C:\\Users\\lairx78\\Desktop\\python'
os.chdir(directory)

In [4]:
# Read the dataframe

df = pd.read_csv('Spam-Classification.csv')
df.iloc[:,0]

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [5]:
# Undersampling

df_ham_undersampling=df[df.iloc[:,0]=='ham'].sample(n = len(df[df.iloc[:,0]=='spam']), random_state = 123)
df_spam=df[df.iloc[:,0]=='spam']
df = pd.concat([df_ham_undersampling, df_spam])
df=df.reset_index(drop=True)

In [6]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,Ha ha - had popped down to the loo when you he...,NaN,NaN,NaN
1,ham,Ever green quote ever told by Jerry in cartoon...,NaN,NaN,NaN
2,ham,Rats. Hey did u ever vote for the next themes?,NaN,NaN,NaN
3,ham,"Sorry I flaked last night, shit's seriously go...",NaN,NaN,NaN
4,ham,Thank you. do you generally date the brothas?,NaN,NaN,NaN
...,...,...,...,...,...
1489,spam,Want explicit SEX in 30 secs? Ring 02073162414...,NaN,NaN,NaN
1490,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,NaN,NaN,NaN
1491,spam,Had your contract mobile 11 Mnths? Latest Moto...,NaN,NaN,NaN
1492,spam,REMINDER FROM O2: To get 2.50 pounds free call...,NaN,NaN,NaN


In [7]:
# Label encoding

label = LabelEncoder().fit_transform(df.iloc[:,0])
label

array([0, 0, 0, ..., 1, 1, 1])

# Data cleaning 

Remove or replace meaningless data using regular expressions 

In [8]:
message=df.iloc[:,1].str.lower().copy()

message=message.str.replace(r'[^ ]+@[^\.]*(\.[a-z]{2,}){1,2}', 'emailaddress', regex=True)
message=message.str.replace(r'(?:http\:\/\/|www.){1}(?:http\:\/\/|www.)?(?![www])[a-zA-Z0-9\-]+(\.[a-zA-Z]{2,3}){1,2}(\/[^/& ]+)*', 'webaddress', regex=True)
message=message.str.replace(r'(£|\$|€|¥|₣|å£){1}\d+(.\d+)?', 'money', regex=True)
message=message.str.replace(r'\b\+?\d{1}[\d\s-]{5,13}\d{1}\b|\b[\d]{4}[\s-]?[\d]{3}[\s-]?[\d]{4}\b', 'phonenumber', regex=True)

# date
# day/month/year
message=message.str.replace(r'\b(3[01]|[12][0-9]|[0]?[1-9]){1}[\ |\-|\/]{1}(1[0-2]|[0]?[1-9]){1}[\ |\-|\/]{1}(\d{2}|\d{4})(?!\d{3})\b', 'date', regex=True)
# year/month/day
message=message.str.replace(r'\b(\d{2}|\d{4})(?!\d{3})[\ |\-|\/]{1}(1[0-2]|[0]?[1-9]){1}[\ |\-|\/]{1}(3[01]|[12][0-9]|[0]?[1-9]){1}\b', 'date', regex=True)
# day/month(english)/year
message=message.str.replace(r'\b(3[01]|[12][0-9]|[0]?[1-9]){1}([\ ]|st|nd|rd|th){1,2}(jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?){1}\b(?:([\ ]|,){0,2}(\d{4}|\d{2}))?\b', 'date', regex=True)
# month(english)/day/year
message=message.str.replace(r'\b(jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?){1}[\ ]?(3[01]|[12][0-9]|[0]?[1-9]){1}((?: )?st|(?: )?nd|(?: )?rd|(?: )?th){0,1}(?:([\ ]|,){0,2}(\d{4}|\d{2}))?\b', 'date', regex=True)


message=message.str.replace(r'\b\d+\b', 'number', regex=True)
message=message.str.replace(r'[^\w\d\s]+', ' ', regex=True)
message=message.str.replace(r'\s+', ' ', regex=True)
message=message.str.replace(r'^\s+|\s+?$', '', regex=True)
  

Remove remove common words like 'a', 'the' using stopwords

In [9]:
common_words=set(stopwords.words('english'))

message=message.apply(lambda m: ' '.join(word for word in m.split() if word not in common_words))

Change the form of words like 'running' to 'run' using Lemmatization

In [10]:
def convert_pos(tag):
    if tag in ['JJ', 'JJR', 'JJS']:
        return wordnet.ADJ
    elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return wordnet.VERB
    elif tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return wordnet.NOUN
    elif tag in ['RB', 'RBR', 'RBS']:
        return wordnet.ADV
    else:
        return wordnet.NOUN

def convert_pos_2(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
for i in range(len(message)):
    tokens = word_tokenize(message[i])
    tagged = pos_tag(tokens)
    
    tokens_list_tokenized = []
    
    for tag in tagged:
        wordnet_pos = convert_pos(tag[1])
        tokens_list_tokenized.append(WordNetLemmatizer().lemmatize(tag[0], pos=wordnet_pos))
    
    tokens_to_string = ' '.join(tokens_list_tokenized)
    message[i]=tokens_to_string

In [12]:
i=1
print('Mail before cleaning: ', df.iloc[i,1].lower(), '\n')
print('Mail after cleaning: ', message[i])

Mail before cleaning:  ever green quote ever told by jerry in cartoon \a person who irritates u always is the one who loves u vry much but fails to express it...!..!! :-) :-) gud nyt" 

Mail after cleaning:  ever green quote ever tell jerry cartoon person irritate u always one love u vry much fails express gud nyt


# Tokenization

Tokenized all the words and choose the 500 most common words

In [13]:
all_words = []

for m in message:
    words = word_tokenize(m)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

print('Total number of Token:', len(all_words))

Total number of Token: 3401


In [14]:
# choose the 500 most common words
word_most_common = [x[0] for x in all_words.most_common(500)]

In [15]:
def only_most_common(message):
    words = word_tokenize(message)
    features = {}
    for word in word_most_common:
        features[word] = (word in words)

    return features

# Model training

There are totally 7 machine learning models, incluing Decision Tree, Random Forest, Logistic Regression, SGD Classifier, Naive Bayes, Support Vector Classifier and Gradient Boosting Classifier

Method 1:

In [16]:
# split data

df_message = list(zip(message, label))
np.random.shuffle(df_message)
df_message_only_most_common = [(only_most_common(text), label) for (text, label) in df_message]
training, test = train_test_split(df_message_only_most_common, test_size=0.2, random_state=1)

print('The number of data for training models: ', len(training), '\n')
print('The number of data for testing models: ', len(test))

The number of data for training models:  1195 

The number of data for testing models:  299


In [17]:
names = ['Decision Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier',
         'Naive Bayes', 'Support Vector Classifier','Gradient Boosting Classifier']

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=100),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel='linear'),
    GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    model_accuracy = accuracy(nltk_model, test)
    print("{} model Accuracy: {}".format(name, model_accuracy))

Decision Tree model Accuracy: 0.939799331103679
Random Forest model Accuracy: 0.9565217391304348
Logistic Regression model Accuracy: 0.9665551839464883
SGD Classifier model Accuracy: 0.9565217391304348
Naive Bayes model Accuracy: 0.9297658862876255
Support Vector Classifier model Accuracy: 0.9698996655518395
Gradient Boosting Classifier model Accuracy: 0.9665551839464883


Method 2:

In [18]:
# split data
df_message_2 = list(zip(message, label))

# shuffle
np.random.shuffle(df_message_2)

message_only_most_common = [only_most_common(text[0]) for text in df_message_2]
list_label = [text[1] for text in df_message_2]

x_training, x_test, y_training, y_test= train_test_split(message_only_most_common, list_label, test_size=0.2, random_state=1)

print('The number of data for training models: ', len(x_training), '\n')
print('The number of data for testing models: ', len(x_test))

x_training = DictVectorizer().fit_transform(x_training)
x_test = DictVectorizer().fit_transform(x_test)

The number of data for training models:  1195 

The number of data for testing models:  299


In [19]:
names = ['Decision Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier',
         'Naive Bayes', 'Support Vector Classifier','Gradient Boosting Classifier']

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=100),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel='linear'),
    GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
]

models = zip(names, classifiers)

for i in range(len(classifiers)):
    nltk_model = classifiers[i]
    nltk_model.fit(x_training, y_training)
    model_accuracy = nltk_model.score(x_test, y_test)
    print("{} model Accuracy: {}".format(names[i], model_accuracy))

Decision Tree model Accuracy: 0.9297658862876255
Random Forest model Accuracy: 0.9632107023411371
Logistic Regression model Accuracy: 0.9698996655518395
SGD Classifier model Accuracy: 0.9698996655518395
Naive Bayes model Accuracy: 0.9163879598662207
Support Vector Classifier model Accuracy: 0.9732441471571907
Gradient Boosting Classifier model Accuracy: 0.9698996655518395


# Ensemble learning

Volting the best prediction by ensemble all the models

In [20]:
names = ['Decision Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier',
         'Naive Bayes', 'Support Vector Classifier','Gradient Boosting Classifier']

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=100),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel='linear'),
    GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
]

models = list(zip(names, classifiers))

ensemble_models =VotingClassifier(estimators=models, voting='hard', n_jobs=1)
ensemble_models.fit(x_training, y_training)
model_accuracy = ensemble_models.score(x_test, y_test)

print('The predicted label: \n', ensemble_models.predict(x_test), '\n')
print("Voting Classifier model Accuracy: {}".format(model_accuracy))

The predicted label: 
 [0 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 0
 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0
 1 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 1 0 0
 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 1
 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 0 1 0 0 1 1
 0 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1
 1 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 0 1 1 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 1 1] 

Voting Classifier model Accuracy: 0.9732441471571907
